In [ ]:
from datetime import datetime
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from markdownify import markdownify 
import folium.plugins as plugins
import folium
import io




# endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """
#defaultView:Map

SELECT DISTINCT ?item ?itemLabel ?location ?locationLabel ?latitude ?longitude ?geo WHERE {


  ?item wdt:P361 wd:Q700216; # Teil von Staatliche Museen zu Berlin
        wdt:P31/wdt:P279* wd:Q33506;
        wdt:P131 ?location.
  
  OPTIONAL { ?item wdt:P625 ?geo } # Get coordinates
  

  SERVICE wikibase:label { bd:serviceParam wikibase:language "de,[AUTO_LANGUAGE]". }

  BIND(geof:latitude(?geo) AS ?latitude).
  BIND(geof:longitude(?geo) AS ?longitude)
}
"""



def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

locations = []

def get_items():

    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else ''
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['locationLabel']['value'] if 'locationLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ''
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude, image_url, item_url, place, alt))






In [20]:
get_items()

m_berlin = folium.Map([52.5097, 13.3675], zoom_start=12)

folium.plugins.Fullscreen(
    position="topright",
    title="Vollbild",
    title_cancel="Schließen",
    force_separate_button=True,
).add_to(m_berlin)

for name, lat, lon, url, item, place, alt in locations:
    if not lat or not lon:  # Überspringen, wenn lat oder lon leer sind
        continue
    
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
        {place}""",
        tooltip=name  # Hover label
    ).add_to(m_berlin)

print("""
[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%0A%0A%20%20%3Fitem%20wdt%3AP361%20wd%3AQ700216%3B%20%23%20Teil%20von%20Staatliche%20Museen%20zu%20Berlin%0A%20%20%20%20%20%20%20%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506.%20%23%20ist%20ein%2Fist%20eine%20Unterklasse%20von%3A%20Museum%0A%20%20%0A%20%20%3Fitem%20wdt%3AP625%20%3Fgeo.%20%20%23%20Get%20coordinates%0A%20%20%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29.%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29%0A%7D%0A)
""")



[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%0A%0A%20%20%3Fitem%20wdt%3AP361%20wd%3AQ700216%3B%20%23%20Teil%20von%20Staatliche%20Museen%20zu%20Berlin%0A%20%20%20%20%20%20%20%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506.%20%23%20ist%20ein%2Fist%20eine%20Unterklasse%20von%3A%20Museum%0A%20%20%0A%20%20%3Fitem%20wdt%3AP625%20%3Fgeo.%20%20%23%20Get%20coordinates%0A%20%20%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29.%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29%0A%7D%0A)



In [21]:

m_berlin


In [22]:
print("""
```{=html}
      </br>
      <table class='table' id='mapTable'>
      <thead>
      <tr>
      <th><b onclick="sortTable(0)" style="color: #56cc9d; cursor:pointer">⭿</b> Name</th>
      <th style='width:20%'><b onclick="sortTable(1)" style="color: #56cc9d; cursor:pointer">⭿</b> Koordinaten</th>
      <th><b onclick="sortTable(2)" style="color: #56cc9d; cursor:pointer">⭿</b> Ort</th>
      </tr>
      </thead>
      <tbody>
```
      """)
for name, lat, lon, url, item, place, alt in locations:
  print(f"""
```{{=html}}
  <tr>
  <td><span style='display:none'>{name}</span><a href='{item}'>{name}</a></td>
```
  """)
  if lat == '' and lon == '':
    print("""
```{=html}
  <td></td>
```
""")
  else:
    print(f"""
```{{=html}}
  <td>{lat},<br> {lon}</td>
```
""")
  print(f"""
```{{=html}}
  <td>{place}</td>
  </tr>
```
  """)
print("""
```{=html}
</tbody></table>
```
""")
print("""
<script>
function sortTable(n) {
  var table, rows, switching, i, x, y, shouldSwitch, dir, switchcount = 0;
  table = document.getElementById("mapTable");
  switching = true;
  // Set the sorting direction to ascending:
  dir = "asc";
  /* Make a loop that will continue until
  no switching has been done: */
  while (switching) {
    // Start by saying: no switching is done:
    switching = false;
    rows = table.rows;
    /* Loop through all table rows (except the
    first, which contains table headers): */
    for (i = 1; i < (rows.length - 1); i++) {
      // Start by saying there should be no switching:
      shouldSwitch = false;
      /* Get the two elements you want to compare,
      one from current row and one from the next: */
      x = rows[i].getElementsByTagName("TD")[n];
      y = rows[i + 1].getElementsByTagName("TD")[n];
      /* Check if the two rows should switch place,
      based on the direction, asc or desc: */
      if (dir == "asc") {
        if (x.innerHTML.toLowerCase() > y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      } else if (dir == "desc") {
        if (x.innerHTML.toLowerCase() < y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      }
    }
    if (shouldSwitch) {
      /* If a switch has been marked, make the switch
      and mark that a switch has been done: */
      rows[i].parentNode.insertBefore(rows[i + 1], rows[i]);
      switching = true;
      // Each time a switch is done, increase this count by 1:
      switchcount ++;
    } else {
      /* If no switching has been done AND the direction is "asc",
      set the direction to "desc" and run the while loop again. */
      if (switchcount == 0 && dir == "asc") {
        dir = "desc";
        switching = true;
      }
    }
  }
}
</script>
""") 


```{=html}
      </br>
      <table class='table' id='mapTable'>
      <thead>
      <tr>
      <th><b onclick="sortTable(0)" style="color: #56cc9d; cursor:pointer">⭿</b> Name</th>
      <th style='width:20%'><b onclick="sortTable(1)" style="color: #56cc9d; cursor:pointer">⭿</b> Koordinaten</th>
      <th><b onclick="sortTable(2)" style="color: #56cc9d; cursor:pointer">⭿</b> Ort</th>
      </tr>
      </thead>
      <tbody>
```
      

```{=html}
  <tr>
  <td><span style='display:none'>Museum für Ostasiatische Kunst</span><a href='http://www.wikidata.org/entity/Q56605294'>Museum für Ostasiatische Kunst</a></td>
```
  

```{=html}
  <td></td>
```


```{=html}
  <td>Berlin</td>
  </tr>
```
  

```{=html}
  <tr>
  <td><span style='display:none'>Kunstbibliothek - Staatliche Museen zu Berlin</span><a href='http://www.wikidata.org/entity/Q6445022'>Kunstbibliothek - Staatliche Museen zu Berlin</a></td>
```
  

```{=html}
  <td>52.50861,<br> 13.36666</td>
```


```{=html}
  <td>Berlin</td>
  